In [1]:
import time
import os
import sys

import torch
from dataloader.transformers import Rescale
from model.lanenet.LaneNet import LaneNet
from torch.utils.data import DataLoader
from torch.autograd import Variable
from torchvision import transforms
from model.utils.cli_helper_test import parse_args
import numpy as np
from PIL import Image
import pandas as pd
import cv2

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def preprocess_frame(frame, transform):
    """웹캠 프레임을 전처리합니다."""
    frame = Image.fromarray(frame)
    frame = transform(frame)
    return frame

def test_with_webcam():
    # 모델 경로 지정
    model_path = r'C:\\Users\\yth12\\Desktop\\LaneNet\\log\\lanenet_DeepLabv3+_CrossEntrophy_epoch100_batchsize8.pth'
    resize_height = 480  # 원하는 이미지 크기로 설정
    resize_width = 640   # 원하는 이미지 크기로 설정

    # Transform 정의
    data_transform = transforms.Compose([
        transforms.Resize((resize_height,  resize_width)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # 모델 로드
    model = LaneNet(arch='DeepLabv3+')  # 모델 타입을 적절히 지정
    state_dict = torch.load(model_path)
    model.load_state_dict(state_dict)
    model.eval()
    model.to(DEVICE)

    # 웹캠 설정
    cap = cv2.VideoCapture(0)  # 웹캠 사용 (카메라 인덱스 0)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Failed to capture image.")
            break

        # 프레임 전처리
        input_img = preprocess_frame(frame, data_transform).to(DEVICE)
        input_img = torch.unsqueeze(input_img, dim=0)

        # 모델 추론
        with torch.no_grad():
            outputs = model(input_img)

        # 결과 후처리
        instance_pred = torch.squeeze(outputs['instance_seg_logits'].detach().to('cpu')).numpy() * 255
        binary_pred = torch.squeeze(outputs['binary_seg_pred']).to('cpu').numpy() * 255

        # 이미지를 numpy 배열로 변환
        instance_pred = np.transpose(instance_pred, (1, 2, 0))

        # 결과 화면에 출력
        cv2.imshow('Webcam Input', frame)
        cv2.imshow('Instance Output', instance_pred.astype(np.uint8))
        cv2.imshow('Binary Output', binary_pred.astype(np.uint8))

        # 'q' 키를 누르면 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 웹캠 릴리즈 및 창 닫기
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    test_with_webcam()


Use DeepLabv3+ as backbone


C:\Users\yth12\AppData\Local\Temp\ipykernel_8044\1676599933.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)
